In [ ]:
%pip install transformers

## Download Data

In [ ]:
%pip install requests_cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.6 MB/s eta 0:00:00


In [ ]:
%run download_dataset

Downloading: 100%|██████████| 36.4M/36.4M [00:00<00:00, 301MB/s]
File downloaded and decompressed successfully.


## Model

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

MODEL_ID = "google-t5/t5-small"
COMP_EMBED_DIM = 512

model = T5ForConditionalGeneration.from_pretrained(MODEL_ID)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
import torch.nn as nn

pooling_layer = nn.Linear(model.encoder.config.d_model, COMP_EMBED_DIM)

## Data

In [ ]:
%pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("text", data_files=["dataset/enwik8"])

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset = dataset["train"]

In [ ]:
import random

sample = random.choice(dataset)
input_ids = tokenizer.encode(text=sample["text"], return_tensors="pt")

In [ ]:
encoder_output = model.encoder(input_ids=input_ids)
hiddens = encoder_output.last_hidden_state

In [ ]:
pooled = pooling_layer(hiddens).mean(dim=-2)

In [ ]:
assert model.decoder.config.d_model == COMP_EMBED_DIM, \
    "Giving the embeddings directly to the decoder"

encoder_hidden_states = pooled.unsqueeze(-2)

decoder_output = model.decoder(
    input_ids=input_ids,
    encoder_hidden_states=encoder_hidden_states,
)

In [ ]:
model_output = model.forward(
    decoder_input_ids=input_ids,
    labels=input_ids,
    encoder_outputs=(encoder_hidden_states,),
)

In [ ]:
model_output.loss

tensor(6.1232, grad_fn=<NllLossBackward0>)